## Welcome to your notebook.


#### requirements

In [1]:
import sys
#!{sys.executable} -m pip install geopandas==0.9.0
#!{sys.executable} -m pip install pynhd
#!{sys.executable} -m pip install cenpy
# forked package to include BG
#!{sys.executable} -m pip install git+https://github.com/jbousquin/cenpy.git
#!{sys.executable} -m pip install git+https://github.com/jbousquin/cenpy.git
!{sys.executable} -m pip install git+https://github.com/walkerke/pygris.git
#!{sys.executable} -m pip install git+https://github.com/jbousquin/pygris.git@multi
#!{sys.executable} -m pip install git+https://github.com/jbousquin/pygris.git@dataGetCensus_50vars
#!conda install --yes --prefix {sys.prefix} descartes
!{sys.executable} -m pip install mapclassify==2.4.3

  Cloning https://github.com/walkerke/pygris.git to /tmp/pip-req-build-hgkgvyo3
  Running command git clone -q https://github.com/walkerke/pygris.git /tmp/pip-req-build-hgkgvyo3
  Resolved https://github.com/walkerke/pygris.git to commit f175d9e2b5145bfa5f7c0579f4ac30dac261d7c3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1 MB 7.7 MB/s eta 0:00:01
     |████████████████████████████████| 7.9 MB 75.9 MB/s eta 0:00:01
  Created wheel for pygris: filename=pygris-0.1.5-py3-none-any.whl size=55392 sha256=2e0112a97dd9fd08b44b28c9c1cd105ff2b316f05e56fa5c122e2f2881522374
  Stored in directory: /tmp/pip-ephem-wheel-cache-f8zziapr/wheels/49/c5/66/3479d734ec8dc40ef4cc35be6f56281981b00d6bb8e311ad52
Successfully built pygris


In [2]:
import pygris
import pandas
import geopandas
from pygris import counties
from pygris.data import get_census
import matplotlib.pyplot as plt

#### Functions

In [3]:
# SoVI dict
def SoVI_variables(subset=None):
    #'QLIMENG': ['B16004_0{}E'.format(str(x).zfill(2)) for x in range(1, 68)],
    x = {'POP': ['B01003_001E'],
         'HH': ['B25003_001E'],
         'QHH200K': ['B19001_001E', 'B19001_017E'],
         'MHHVAL': ['B25077_001E'],
         'MGRENT': ['B25064_001E'],
         'PCINC': ['B19301_001E'],
         'QASIAN': ['B02001_001E', 'B02001_005E'],
         'QBLACK': ['B02001_001E', 'B02001_003E'],
         'QFHHLD': ['B09019_003E', 'B09019_008E', 'B09019_009E'],
         'QPOV': ['B17021_001E', 'B17021_002E'],
         'QNOED': ['B15003_001E',
                   'B15003_017E', 'B15003_018E',
                   'B15003_019E', 'B15003_020E',
                   'B15003_021E', 'B15003_022E',
                   'B15003_023E', 'B15003_024E',
                   'B15003_025E'],
         'QNOVEH': ['B25044_001E', 'B25044_003E', 'B25044_010E'],
         'QFAM': ['B11003_001E', 'B11003_003E'],
         'MAGE': ['B01002_001E'],
         'QSSBEN': ['B19055_001E', 'B19055_002E'],
         'QAGEDEP': ['B01001_001E', 'B01001_003E', 'B01001_027E',
                     'B01001_020E', 'B01001_021E',
                     'B01001_022E', 'B01001_023E',
                     'B01001_024E', 'B01001_025E',
                     'B01001_044E', 'B01001_045E',
                     'B01001_046E', 'B01001_047E',
                     'B01001_048E', 'B01001_049E'],
         'QUNOCC': ['B25002_001E', 'B25002_003E'],
         'QRENT': ['B25008_001E', 'B25008_003E'],
         'QHISP': ['B03003_001E', 'B03003_003E'],
         'QLIMENG': ['B16004_001E',
                     'B16004_007E', 'B16004_008E', 'B16004_012E', 'B16004_013E', 'B16004_017E', 'B16004_018E', 'B16004_022E', 'B16004_023E', 
                     'B16004_029E', 'B16004_030E', 'B16004_034E', 'B16004_035E', 'B16004_039E', 'B16004_040E', 'B16004_044E', 'B16004_045E', 
                     'B16004_051E', 'B16004_052E', 'B16004_056E', 'B16004_057E', 'B16004_061E', 'B16004_062E', 'B16004_066E', 'B16004_067E', 
                    ],
         'QNOINS': ['B27010_001E', 
                    'B27010_017E', 'B27010_033E', 'B27010_050E', 'B27010_066E'],
         'HHCROWD': ['B25014_001E',
                     'B25014_006E', 'B25014_007E', 'B25014_012E', 'B25014_013E'],
         'QHHBURD': ['B25091_001E',
                     'B25091_008E', 'B25091_009E', 'B25091_010E', 'B25091_011E',
                     'B25091_019E', 'B25091_020E', 'B25091_021E', 'B25091_022E'],
         'QUNEMP': ['B23025_001E', 'B23025_005E'],
         'QNATIVE': ['B02001_001E', 'B02001_004E'],
         'QFEM': ['B01001_001E', 'B01001_026E']
        }
    if subset:
        if subset in ['keys', 'indicators']:
            return list(x.keys())
        else:
            return x[subset]
    else:
        # Flatten
        return list(set([val for sublist in list(x.values()) for val in sublist]))

In [4]:
# SVI dict
def SVI_variables(subset=None):
    x = {'FIPS': [],
         'Area': [],
         'TotPop': ['B01003_001E'],
         'HousUnits': ['B25001_001E'],
         'Household': ['B11001_001E'],
         'Poverty150': ['B29003_001E', 'B29003_002E'],
         'Unemploy': ['B23025_001E', 'B23025_005E'],
         'HouseBurd': ['B25074_001E', 'B25074_006E', 'B25074_007E',
                       'B25074_008E', 'B25074_009E', 'B25074_015E',
                       'B25074_016E', 'B25074_017E', 'B25074_018E',
                       'B25074_024E', 'B25074_025E', 'B25074_026E',
                       'B25074_027E', 'B25074_033E', 'B25074_034E',
                       'B25074_035E', 'B25074_036E', 'B25074_042E',
                       'B25074_043E', 'B25074_044E', 'B25074_045E',
                       'B25091_001E', 'B25091_008E', 'B25091_009E',
                       'B25091_010E', 'B25091_011E', 'B25091_019E',
                       'B25091_020E', 'B25091_021E', 'B25091_022E'],
         'NoHSDiplo': ['B15003_001E', 'B15003_017E', 'B15003_018E',
                       'B15003_019E', 'B15003_020E', 'B15003_021E',
                       'B15003_022E', 'B15003_023E', 'B15003_024E',
                       'B15003_025E'],
         'NoHlthIns': ['B27010_001E', 'B27010_017E', 'B27010_033E',
                       'B27010_050E', 'B27010_066E'],
         '65andover': ['B27010_001E', 'B27010_051E'],
         '17andbelow': ['B29001_001E'],
         'DisableCiv': ['C21007_001E', 'C21007_005E', 'C21007_008E',
                        'C21007_012E', 'C21007_015E', 'C21007_020E',
                        'C21007_023E', 'C21007_027E', 'C21007_030E'],
         'SPH': ['B11001_002E', 'B11001_005E', 'B11001_006E'],
         'ELP': ['B16004_001E', 'B16004_007E', 'B16004_008E', 'B16004_012E',
                 'B16004_013E', 'B16004_017E', 'B16004_018E', 'B16004_022E',
                 'B16004_023E', 'B16004_029E', 'B16004_030E', 'B16004_034E',
                 'B16004_035E', 'B16004_039E', 'B16004_040E', 'B16004_044E',
                 'B16004_045E', 'B16004_051E', 'B16004_052E', 'B16004_056E',
                 'B16004_057E', 'B16004_061E', 'B16004_062E', 'B16004_066E',
                 'B16004_067E',],
         'Hisp': ['B03002_001E', 'B03002_012E'],
         'Black': ['B03002_001E', 'B03002_004E'],
         'Asian': ['B03002_001E', 'B03002_006E'],
         'AIAN': ['B03002_001E', 'B03002_005E'],
         'NHPI': ['B03002_001E', 'B03002_007E'],
         'TwoRace': ['B03002_001E', 'B03002_009E'],
         'OtherRace': ['B03002_001E', 'B03002_008E'],
         'MUStruct': ['B25024_001E', 'B25024_007E',
                      'B25024_008E', 'B25024_009E'],
         'MobHome': ['B25024_001E', 'B25024_010E'],
         'Crowd': ['B25014_001E', 'B25014_005E', 'B25014_006E', 'B25014_007E',
                   'B25014_011E', 'B25014_012E', 'B25014_013E'], 
         'NoVeh': ['B25044_001E', 'B25044_003E', 'B25044_010E'],
         'GrpQuarter': ['B09019_002E', 'B09019_026E'],
         }

    if subset:
        if subset in ['keys', 'indicators']:
            return list(x.keys())
        else:
            return x[subset]
    else:
        # Flatten
        return list(set([val for sublst in list(x.values()) for val in sublst]))


def SVI_indicators(subset=None):
    x = {'Base_Data': ['FIPS', 'Area', 'TotPop', 'HousUnits', 'Household'],
         'Socioeconomic_Status': ['Poverty150', 'Unemploy', 'HouseBurd',
                                  'NoHSDiplo', 'NoHlthIns'],
         'Household Characteristics': ['65andover', '17andbelow', 'DisableCiv',
                                       'SPH', 'ELP'],
         'Racial_and_Ethnic_Minority Status': ['Hisp', 'Black', 'Asian',
                                               'AIAN', 'NHPI', 'TwoRace',
                                               'OtherRace',],
         'Housing_Type_and_Transportation': ['MUStruct', 'MobHome', 'Crowd',
                                             'NoVeh', 'GrpQuarter'],
         }
    if subset:
        return x[subset] 
    return x

In [5]:
def SoVI_preprocess(df_in):
    # Deep copy table
    df = df_in.copy()
    # List new column names
    cols = SVI_variables('keys')
    
    # cols maths: just rename, percents where col2/col1, and difs where col2-col1/col1
    renames = ['POP', 'HH', 'MHHVAL', 'MGRENT', 'PCINC', 'MAGE']
    pct = ['QHH200K', 'QASIAN', 'QBLACK', 'QPOV',
           'QFAM', 'QSSBEN', 'QUNOCC', 'QRENT',
           'QHISP', 'QLIMENG', 'QNOINS', 'HHCROWD',
           'QHHBURD', 'QUNEMP', 'QNATIVE', 'QFEM',
           'QFHHLD', 'QNOVEH', 'QAGEDEP']
    pct_dif = ['QNOED']

    for col in cols:
        metric_cols = SVI_variables(col)
        if col in renames:
            df = df.rename(columns={metric_cols[0]: col})
        elif col in pct:
            if len(metric_cols)>2:
                print(col)
                # Numerator is sum of list (excluding denominator)
                df[col] = df[metric_cols[1:]].sum(axis=1).divide(df[metric_cols[0]]) *100.0
            else:
                # Numerator is last in list
                df[col] = df[metric_cols[1]].divide(df[metric_cols[0]]) *100.0
                    
        elif col in pct_dif:
            df[col] = (df[metric_cols[0]] - df[metric_cols[1:]].sum(axis=1)).divide(df[metric_cols[0]]) *100.0
        else:
            # TODO: throw error
            print('{} has no preprocessing assigned'.format(col))
    #TODO: drop original cols?
    
    return df

In [6]:
def SVI_preprocess(df_in):
    # Deep copy table
    df = df_in.copy()
    # List new column names
    #indicators = SVI_variables('keys')

    # cols maths: just rename, percents where col2/col1,
    # and difs where col2-col1/col1
    renames = ['TotPop', 'HousUnits', 'Household']
    
    # Percent of population ([0] is TotPop)
    pct_pop = ['Poverty150', 'Unemploy', 'Hisp', 'Black', 'Asian', 'AIAN',
               'NHPI', 'TwoRace', 'OtherRace', 'ELP']
    # sum/[0] but not over pop
    pct = ['65andover', 'NoHSDiplo', 'NoHlthIns', 'DisableCiv', 'SPH',
           'MUStruct', 'MobHome', 'Crowd', 'NoVeh', 'GrpQuarter']
    pct += pct_pop  # Combine for now
    
    pct_dif = ['17andbelow']

    for col in SVI_variables('keys'):
        metric_cols = SVI_variables(col)
        print(col)
        if col in renames:
            df = df.rename(columns={metric_cols[0]: col})
        elif col in pct:
            if len(metric_cols)>2:
                # Numerator is sum of list (excluding denominator)
                df[col] = df[metric_cols[1:]].sum(axis=1).divide(df[metric_cols[0]]) *100.0
            else:
                # Numerator is last in list
                df[col] = df[metric_cols[1]].divide(df[metric_cols[0]]) *100.0
                    
        elif col in pct_dif:
            df[col] = (df['TotPop'] - df[metric_cols[0]]).divide(df['TotPop']) *100.0
        elif col == 'HouseBurd':
            renter = df[metric_cols[1:21]].sum(axis=1).divide(df[metric_cols[0]])
            owner = df[metric_cols[22:]].sum(axis=1).divide(df[metric_cols[21]])
            df[col] = renter + owner
        else:
            # TODO: throw error
            print('{} has no preprocessing assigned'.format(col))
    #TODO: drop original cols?
    
    return df

In [7]:
import numpy as np
import pandas as pd
import requests

#### Retrieve geometry and data using county name

In [ ]:
# Query geometry by name
geo_sussex = pygris.block_groups(state = "DE", county = "Sussex", year = 2021)
geo_sussex

In [ ]:
# Query population results for block groups in above county
df_sussex = get_census(dataset = "acs/acs5",
                       variables = 'B01001_001E',
                       year=2021,
                       params = {
                                 "for": "block group:*",
                                 "in": "state:10;county:005;tract:*"
                                 }, 
                       return_geoid = True)

In [ ]:
# join population to geometry
gdf_sussex = geo_sussex.join(df_sussex.set_index('GEOID'), on='GEOID')

In [ ]:
# Plot it
fig, ax = plt.subplots(1,1,figsize=(20,20))
gdf_sussex.dropna(subset=['B01001_001E'], axis=0).plot('B01001_001E', ax=ax, cmap='plasma', legend=True)

#### Retrieve geometry and data using AOI bounding box

In [ ]:
# Define AOI
aoi = '/arcgis/home/Pens_Perd_Watershed_Dissolve.shp'
aoi_gdf = geopandas.read_file(aoi)

In [ ]:
# Use hydro geometry to get census county geometries
hydro_counties = pygris.counties(subset_by=tuple(aoi_gdf.total_bounds.tolist()))
hydro_counties

In [ ]:
# Plot it
hydro_counties.plot()

In [ ]:
# loop over state county combinations to query block groups geo and data
bgs=[]
for state in set(hydro_counties['STATEFP']):
    county_FIPS = list(hydro_counties[hydro_counties['STATEFP']==state]['COUNTYFP'])
    # Get block group geos
    bgs_geos = pygris.block_groups(state = state, county = county_FIPS, year = 2021)
    # Get data
    bgs_data = get_census(dataset = "acs/acs5",
                          variables = 'B01001_001E',
                          year=2021,
                          params = {
                                    "for": "block group:*",
                                    "in": "state:{};county:{};tract:*".format(state, ','.join(county_FIPS))
                                    }, 
                          return_geoid = True)
    # Join and append to running list
    bgs.append(bgs_geos.join(bgs_data.set_index('GEOID'), on='GEOID'))
gdf = pandas.concat((bgs), sort=False, axis=0)
gdf

In [ ]:
# Plot it
fig, ax = plt.subplots(1,1,figsize=(20,20))
gdf.dropna(subset=['B01001_001E'], axis=0).plot('B01001_001E', ax=ax, cmap='plasma', legend=True)

###### Test Case 1

From: https://github.com/walkerke/tidycensus/pull/165

In [ ]:
variables = ["B01001_003E", "B01001_004E", "B01001_005E", "B01001_006E", "B01001_007E",
             "B01001_008E", "B01001_009E", "B01001_010E", "B01001_011E", "B01001_012E", 
             "B01001_013E", "B01001_014E", "B01001_015E", "B01001_016E", "B01001_017E", 
             "B01001_018E", "B01001_019E", "B01001_020E", "B01001_021E", "B01001_022E", 
             "B01001_023E", "B01001_024E", "B01001_025E", "B01001_026E", "B25002_002E",
             "B03003_003E"]

In [ ]:
test1 = get_census(dataset = "acs/acs5",
                   variables = "B03003_003E",
                   year = 2017,
                   params = {
                             "for": "tract:*",
                             "in": "state:36;county:*",
                            },
                   return_geoid = True)

#    geography = "tract", year = 2017, variables = "B03003_003", 
#                 output = "wide", state = 36, geometry = F, key = api_key)

In [ ]:
test2 = get_census(dataset = "acs/acs5",
                   variables = variables,
                   year = 2017,
                   params = {
                             "for": "tract:*",
                             "in": "state:36;county:*",
                            },
                   return_geoid = True)

#test2 = get_census(geography = "tract", year = 2017, variables = variables,
#                 output = "wide", state = 36, geometry = F, key = api_key)

In [ ]:
test1[test1['GEOID'] == "36047037500"]["B03003_003E"] == test2[test2['GEOID'] == "36047037500"]["B03003_003E"]

In [ ]:
test3 = test1.merge(test2, left_index=True, right_index=True, how='left', indicator=True)
assert len(test3[test3['_merge'] == 'both']) == len(test3), 'Batch index mis-match'

###### Test Case 2

From: https://github.com/hrecht/censusapi/issues/82

In [ ]:
#test_82 = get_census(dataset = "acs/acs5",
#                     variables = ["B01001H_007E", "B01001H_022E"],
#                     year = 2014,
#                     params = {
#                               "for": "tract:*",
#                               "in": "state:*;county:*",
#                              },
#                     return_geoid = True)

#getCensus(
#        name = "acs/acs5",
#        vintage = "2014",
#        vars = c("B01001H_007E", "B01001H_022E"),
#        region = "zip code tabulation area:*",
#        regionin = "state:*"
#    ) %>%
#    filter(zip_code_tabulation_area == "07980")

In [ ]:
# Group B01001 (001-049E)
estimates = ['0'+ str(z) for z in range(1, 10)]
estimates +=list(range(10, 50))
group_B01001 = ['B01001_0'+ str(v) + 'E' for v in estimates]
#group_B01001

In [ ]:
acs_pop_group = get_census(dataset = "acs/acs5",
                           variables = group_B01001,
                           year = 2017,
                           params = {
                               "for": "tract:*",
                               "in": "state:02;county:*",
                           },
                           return_geoid = True)

#acs_pop_group <- getCensus(
#    name = "acs/acs5",
#    vintage = 2017,
#    vars = "group(B01001)",
#    region = "tract:*",
#    regionin = "state:02")

In [ ]:
acs_pop_manual = get_census(dataset = "acs/acs5",
                            variables = 'B01001_001E',
                            year = 2017,
                            params = {
                                "for": "tract:*",
                                "in": "state:02;county:*",
                            },
                            return_geoid = True)
#acs_pop_manual <- getCensus(
#    name = "acs/acs5",
#    vintage = 2017,
#    vars = c("NAME", "GEO_ID", group_B01001$name),
#    region = "tract:*",
#    regionin = "state:02")

In [ ]:
comp = acs_pop_group['B01001_001E']== acs_pop_manual['B01001_001E']
comp.value_counts()

In [ ]:
test_acs_pop_group = acs_pop_group.merge(acs_pop_manual, left_index=True, right_index=True, how='left', indicator=True)
assert len(test3[test3['_merge'] == 'both']) == len(test3), 'Batch index mis-match'

###### Back to main but with full set of vars

Strip down data.py

In [ ]:
import numpy as np
import pandas as pd
import requests

In [ ]:
endpoint = "https://api.census.gov/data"
dataset='acs/acs5'
variables = SVI_variables()
year=2021
base = f"{endpoint}/{year}/{dataset}"

In [ ]:
state = list(set(hydro_counties['STATEFP']))[1]
county_FIPS = list(hydro_counties[hydro_counties['STATEFP']==state]['COUNTYFP'])
county_FIPS = [county_FIPS[0]]
print(state +': ' + ','.join(county_FIPS))

In [ ]:
params = {
          "for": "block group:*",
          "in": "state:{};county:{};tract:*".format(state, ','.join(county_FIPS))
          }

In [ ]:
data=[]
n_chunks = np.ceil(len(variables) / 50)

In [ ]:
idx = 2

In [ ]:
# Get results
chunk = np.array_split(variables, n_chunks)[idx]
joined_vars = ",".join(chunk)
params.update({'get': joined_vars})
req = requests.get(url = base, params = params)
if req.status_code != 200:
    raise SyntaxError(f"Request failed. The Census Bureau error message is {req.text}")
df = pd.DataFrame().from_records(req.json()[1:], columns=req.json()[0])

In [ ]:
#data[0]
df

In [ ]:
#data[1]
df

In [ ]:
#data[2]
df

In [ ]:
for chunk in np.array_split(variables, n_chunks):
    joined_vars = ",".join(chunk)
    params.update({'get': joined_vars})
    req = requests.get(url = base, params = params)
    if req.status_code != 200:
        raise SyntaxError(f"Request failed. The Census Bureau error message is {req.text}")
    out = pd.DataFrame().from_records(req.json()[1:], columns=req.json()[0])
    data+=[out]

Run return geoid on loop

In [ ]:
my_cols = list(df.columns)
assert 'state' in my_cols, 'Problem'
state_ix = my_cols.index("state")
geoid_cols = my_cols[state_ix:]
print(geoid_cols)
df['GEOID'] = df[geoid_cols].agg("".join, axis = 1)
df.drop(geoid_cols, axis = 1, inplace= True)

In [ ]:
data+=[df]

In [ ]:
data[0]

In [ ]:
data[1]

In [ ]:
data[2]

In [ ]:
for var_df in data:
    my_cols = list(var_df.columns)
    assert 'state' in my_cols, 'Problem'
    state_ix = my_cols.index("state")
    geoid_cols = my_cols[state_ix:]
    print(geoid_cols)
    var_df['GEOID'] = var_df[geoid_cols].agg("".join, axis = 1)
    var_df.drop(geoid_cols, axis = 1, inplace= True)

Concat list of results

In [ ]:
out = pd.concat((data), sort=False, axis=1)
out

In [ ]:
out['GEOID']

In [ ]:
# NOTE: no GEOID
my_cols = out.columns.to_list()
my_cols

In [ ]:
print(state + ': ' + ','.join(county_FIPS))

In [ ]:
# Have to drop non-unique columns (done based on return_geoid method)

##### Run it in problem county using local function

In [ ]:
state = list(set(hydro_counties['STATEFP']))[0]
county_FIPS = list(hydro_counties[hydro_counties['STATEFP']==state]['COUNTYFP'])
print(state +': ' + ','.join(county_FIPS))

In [ ]:
county_FIPS = [county_FIPS[0]]
county_FIPS

In [ ]:
# Get data
bgs_data = local_func(dataset = "acs/acs5",
                      variables = SVI_variables(),
                      year=2021,
                      params = {
                                "for": "block group:*",
                                "in": "state:{};county:{};tract:*".format(state, ','.join(county_FIPS))
                                }, 
                      return_geoid = True)

In [ ]:
bgs_data

In [ ]:
bgs_data['GEOID']

##### Run it over the set of geometries now (just state of AL)

return_geoid = True works for FL (12) but not AL (01) county '097' (works for '003')

In [ ]:
state = list(set(hydro_counties['STATEFP']))[0]
county_FIPS = list(hydro_counties[hydro_counties['STATEFP']==state]['COUNTYFP'])
print(state +': ' + ','.join(county_FIPS))

In [ ]:
county_FIPS = [county_FIPS[0]]
county_FIPS

In [ ]:
# Get block group geos
bgs_geos = pygris.block_groups(state = state, county = county_FIPS, year = 2021)
bgs_geos

In [ ]:
# Get data
bgs_data = get_census(dataset = "acs/acs5",
                      variables = SVI_variables(),
                      year=2021,
                      params = {
                                "for": "block group:*",
                                "in": "state:{};county:{};tract:*".format(state, ','.join(county_FIPS))
                                }, 
                      return_geoid = True)

In [ ]:
bgs_data

In [ ]:
# loop over state county combinations to query block groups geo and data
bgs1=[]
for state in set(hydro_counties['STATEFP']):
    county_FIPS = list(hydro_counties[hydro_counties['STATEFP']==state]['COUNTYFP'])
    # Get block group geos
    bgs_geos = pygris.block_groups(state = state, county = county_FIPS, year = 2021)
    # Get data
    bgs_data = local_func(dataset = "acs/acs5",
                          variables = SVI_variables(),
                          year=2021,
                          params = {
                                    "for": "block group:*",
                                    "in": "state:{};county:{};tract:*".format(state, ','.join(county_FIPS))
                                    }, 
                          return_geoid = True,
                          guess_dtypes = True,
                         )
    # Join and append to running list
    joined_gdf = bgs_geos.join(bgs_data.set_index('GEOID'), on='GEOID')
    bgs1.append(joined_gdf)
    #bgs1.append(bgs_data)
gdf1 = pandas.concat((bgs1), sort=False, axis=0)
gdf1

##### Run it over the full set of geometries now (AL & FL)

In [ ]:
# loop over state county combinations to query block groups geo and data
bgs=[]
for state in set(hydro_counties['STATEFP']):
    county_FIPS = list(hydro_counties[hydro_counties['STATEFP']==state]['COUNTYFP'])
    # Get block group geos
    bgs_geos = pygris.block_groups(state = state, county = county_FIPS, year = 2021)
    # Get data
    bgs_data = get_census(dataset = "acs/acs5",
                          variables = SVI_variables(),
                          year=2021,
                          params = {
                                    "for": "block group:*",
                                    "in": "state:{};county:{};tract:*".format(state, ','.join(county_FIPS))
                                    }, 
                          return_geoid = True,
                          guess_dtypes = True,
                         )
    # Join and append to running list
    bgs.append(bgs_geos.join(bgs_data.set_index('GEOID'), on='GEOID'))
gdf = pandas.concat((bgs), sort=False, axis=0)
gdf

In [ ]:
# Plot it
fig, ax = plt.subplots(1,1,figsize=(20,20))
gdf.dropna(subset=['B01001_001E'], axis=0).plot('B01001_001E', ax=ax, cmap='plasma', legend=True)

Show it works without geoid

In [ ]:
# loop over state county combinations to query block groups geo and data
bgs1_no_geo = []
for state in set(hydro_counties['STATEFP']):
    county_FIPS = list(hydro_counties[hydro_counties['STATEFP']==state]['COUNTYFP'])
    # Get data
    bgs_data = get_census(dataset = "acs/acs5",
                          variables = SVI_variables(),
                          year=2021,
                          params = {
                                    "for": "block group:*",
                                    "in": "state:{};county:{};tract:*".format(state, ','.join(county_FIPS))
                                    }, 
                          return_geoid = False)
    # Append to running list
    bgs1_no_geo.append(bgs_data)
df_no_geo = pandas.concat((bgs1_no_geo), sort=False, axis=0)
df_no_geo

###### SVI processing

In [ ]:
svi_results = SVI_preprocess(gdf)

In [ ]:
svi_results

In [ ]:
# reduce cols
#dropped     'state',
#             'county',
#             'tract',
#             'block group',
#             'NAME',
keep_cols = ['GEOID',
             'QHH200K',
             'QASIAN',
             'QBLACK',
             'QFHHLD',
             'QPOV',
             'QNOED',
             'QNOVEH',
             'QFAM',
             'QSSBEN',
             'QAGEDEP',
             'QUNOCC',
             'QRENT',
             'QHISP',
             'QLIMENG',
             'QNOINS',
             'HHCROWD',
             'QHHBURD',
             'QUNEMP',
             'QNATIVE',
             'QFEM']
df_svi_out = svi_results[keep_cols]

In [ ]:
df_svi_out

Write out results

In [ ]:
# Export to excel
df_svi_out.to_csv('/arcgis/home/SVI_2021_PPBEP.csv')

In [ ]:
dataset = '/arcgis/home/SVI_2021_PPBEP.csv'

###### Run AOI for all SVI variables

In [ ]:
# loop over state county combinations to query block groups geo and data
bgs=[]
for state in set(hydro_counties['STATEFP']):
    county_FIPS = list(hydro_counties[hydro_counties['STATEFP']==state]['COUNTYFP'])
    # Get block group geos
    #bgs_geos = pygris.block_groups(state = state, county = county_FIPS, year = 2021)
    # Get data
    bgs_data = get_census(dataset = "acs/acs5",
                          variables = SVI_variables(),
                          year=2021,
                          params = {
                                    "for": "block group:*",
                                    "in": "state:{};county:{};tract:*".format(state, ','.join(county_FIPS))
                                    }, 
                          return_geoid = True)
    # Join and append to running list
    #bgs.append(bgs_geos.join(bgs_data.set_index('GEOID'), on='GEOID'))
    bgs.append(bgs_data)
df2 = pandas.concat((bgs), sort=False, axis=0)
df2

In [ ]:
df2

In [ ]:
gdf2['B01001_001E'].dropna()

In [ ]:
len(SVI_variables())

In [ ]:
# Same population plot
fig, ax = plt.subplots(1,1,figsize=(20,20))
gdf2.dropna(subset=['B01001_001E'], axis=0).plot('B01001_001E', ax=ax, cmap='plasma', legend=True)

###### Retrieve geometry and data using AOI bounding box 2

In [ ]:
import cenpy

# get specific ACS vintage
acs_2021 = cenpy.products.ACS(2021)

In [ ]:
# Use hydro geometry to get census geometry and data
geoms, data = acs_2021._from_bbox(bounding_box=aoi_cats.total_bounds,
                                  level='blockgroup',
                                  variables=SVI_variables())

In [ ]:
# Join dfs
#TODO: join on state/county/tract instead of GEOID?
data['GEOID'] = data['state'] + data['county'] + data['tract'] + data['block group']
sf = geoms.join(data.set_index('GEOID'), on='GEOID')
sf = sf.reset_index()

In [ ]:
# Process the retrieved values
df_hydro = SVI_preprocess(sf)

In [ ]:
# Plot results
fig, ax = plt.subplots(1,1,figsize=(20,20))
df_hydro.dropna(subset=['POP'], axis=0).plot('POP', ax=ax, cmap='plasma', legend=True)

#### SVI tests (Escambia FL) 12-033 block group

In [8]:
# Get block group geos
bgs_geos_033 = pygris.block_groups(state = 'FL', county = '033', year = 2021)

Using FIPS code '12' for input 'FL'


ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


In [9]:
# Get SVI variables
bgs_data_033 = get_census(dataset = "acs/acs5",
                          variables = SVI_variables(),
                          year=2021,
                          params = {
                                    "for": "block group:*",
                                    "in": "state:{};county:{};tract:*".format('12', '033')
                                    },
                          return_geoid = True,
                          guess_dtypes = True,
                         )

In [ ]:
# Preprocess SVI variables
svi_results_033 = SVI_preprocess(bgs_data_033)

In [ ]:
# Combine with geos
result_033_2021 = bgs_geos_033.merge(svi_results_033, on='GEOID')

In [ ]:
# Export for comparison
result_033_2021.to_file('/arcgis/home/SVI_2021_Escambia_FL.shp')

In [ ]:
# export without geometry
result_033_2021_csv = result_033_2021.drop(columns=['geometry'])
result_033_2021_csv.to_csv('/arcgis/home/SVI_2021_Escambia_FL.csv')

#### SVI tests (Escambia FL) 12-033 Tract

In [18]:
# Get track geos (2020)
tract_geos_033 = pygris.tracts(state = 'FL', county = '033', year = 2020)

Using FIPS code '12' for input 'FL'


In [19]:
# Get SVI variables
tract_data_033 = get_census(dataset = "acs/acs5",
                            variables = SVI_variables(),
                            year=2020,
                            params = {
                                    "for": "tract:*",
                                    "in": "state:{};county:{}".format('12', '033')
                                    },
                            return_geoid = True,
                            guess_dtypes = True,
                           )

In [20]:
# Preprocess SVI variables
svi_results_033 = SVI_preprocess(tract_data_033)

FIPS
FIPS has no preprocessing assigned
Area
Area has no preprocessing assigned
TotPop
HousUnits
Household
Poverty150
Unemploy
HouseBurd
NoHSDiplo
NoHlthIns
65andover
17andbelow
DisableCiv
SPH
ELP
Hisp
Black
Asian
AIAN
NHPI
TwoRace
OtherRace
MUStruct
MobHome
Crowd
NoVeh
GrpQuarter


In [21]:
# Combine with geos
result_033_2020 = tract_geos_033.merge(svi_results_033, on='GEOID')

In [22]:
# Export for comparison
result_033_2020.to_file('/arcgis/home/SVI_2020_Escambia_FL.shp')

/tmp/ipykernel_19/1705729791.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  result_033_2020.to_file('/arcgis/home/SVI_2020_Escambia_FL.shp')


In [23]:
# export without geometry
result_033_2020_csv = result_033_2020.drop(columns=['geometry'])
result_033_2020_csv.to_csv('/arcgis/home/SVI_2020_Escambia_FL.csv')

In [27]:
# looking for denominators for renter/owner

In [25]:
metric_cols = SVI_variables('HouseBurd')
metric_cols[0]

'B25074_001E'

In [26]:
metric_cols[21]

'B25091_001E'

#### SoVI tests (FL counties)

In [ ]:
import cenpy

# get specific ACS vintage
acs = cenpy.products.ACS(2019)

In [ ]:
# counties in FL
FL_counties = ['BAY',
               'CALHOUN',
               'FRANKLIN',
               'GADSDEN',
               'GULF',
               'JACKSON',
               'LEON',
               'LIBERTY',
               'WAKULLA',
               'WALTON',
               'WASHINGTON']

In [ ]:
df = None
for county in FL_counties:
    county_str = '{}, {}'.format(county, 'FL')
    df_temp = acs.from_county(county_str,
                              level='blockgroup',
                              variables=SVI_variables())
    if df is None:
        df = df_temp
    else:
        df = df.append(df_temp)

In [ ]:
# Run calcs
df2 = SVI_preprocess(df)
#strip geom

In [ ]:
# reduce cols
keep_cols = ['GEOID',
             'state',
             'county',
             'tract',
             'block group',
             'NAME',
             'QHH200K',
             'QASIAN',
             'QBLACK',
             'QFHHLD',
             'QPOV',
             'QNOED',
             'QNOVEH',
             'QFAM',
             'QSSBEN',
             'QAGEDEP',
             'QUNOCC',
             'QRENT',
             'QHISP',
             'QLIMENG',
             'QNOINS',
             'HHCROWD',
             'QHHBURD',
             'QUNEMP',
             'QNATIVE',
             'QFEM']
df_out = df2[keep_cols]

In [ ]:
# Export to excel
#df_out.to_csv('/arcgis/home/SVI_2019.csv')